# multi-hot encode

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("train_emoji.csv")
test = pd.read_csv("tesss.csv")
x_train = train.iloc[: , 0]
y_train = train.iloc[: , 1]
x_test = test.iloc[: , 0]
y_test = test.iloc[: , 1]

In [3]:
all_texts = x_train.tolist() + x_test.tolist()

vocab = {}
for text in all_texts:
    for word in text.split():
        if word not in vocab:
            vocab[word] = len(vocab)

num_words = len(vocab)
print(num_words)

333


In [4]:

def multi_hot_encode(text, vocab):
    words = text.split()
    multi_hot_vector = np.zeros(num_words)
    for word in words:
        if word in vocab:
            index = vocab[word]
            multi_hot_vector[index] = 1
    return multi_hot_vector

# انکود کردن داده‌های آموزش و تست
x_train = np.array([multi_hot_encode(text, vocab) for text in x_train])
x_test = np.array([multi_hot_encode(text, vocab) for text in x_test])
print(x_train)
print(len(x_train[0]))


[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
333


In [5]:
import numpy as np

# Example data (replace with your actual data)
y_train = np.array(y_train.tolist())  # Example of training labels
y_test = np.array(y_test.tolist())        # Example of test labels

num_emojis = 5  # Number of classes (emojis)

def one_hot_encode(index, num_classes):
    one_hot_vector = np.zeros(num_classes)
    one_hot_vector[int(index)] = 1  # Ensure index is cast to int
    return one_hot_vector

# Ensure y_train and y_test are of integer type
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Encode training labels
y_train = np.array([one_hot_encode(label, num_emojis) for label in y_train])

# Encode test labels (convert to list first if needed)
y_test = np.array([one_hot_encode(label, num_emojis) for label in y_test])




In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

input_dim = num_words
output_dim = num_emojis

# ساخت مدل
model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim, activation='softmax'))

# کامپایل کردن مدل
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# نمایش خلاصه‌ای از مدل
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               42752     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 51,333
Trainable params: 51,333
Non-trainable params: 0
_________________________________________________________________


In [7]:
# آموزش مدل
history = model.fit(x_train ,y_train , epochs=10,
          batch_size=2
         , validation_split=0.2)

# ارزیابی مدل


Epoch 1/10
52/52 [==============================] - 2s 17ms/step - loss: 1.6011 - accuracy: 0.2115 - val_loss: 1.6184 - val_accuracy: 0.1852
Epoch 2/10
52/52 [==============================] - 0s 5ms/step - loss: 1.5211 - accuracy: 0.3365 - val_loss: 1.6270 - val_accuracy: 0.2222
Epoch 3/10
52/52 [==============================] - 0s 5ms/step - loss: 1.4432 - accuracy: 0.3942 - val_loss: 1.6248 - val_accuracy: 0.1852
Epoch 4/10
52/52 [==============================] - 0s 5ms/step - loss: 1.3642 - accuracy: 0.4327 - val_loss: 1.6340 - val_accuracy: 0.2222
Epoch 5/10
52/52 [==============================] - 0s 5ms/step - loss: 1.2815 - accuracy: 0.5385 - val_loss: 1.6393 - val_accuracy: 0.2593
Epoch 6/10
52/52 [==============================] - 0s 5ms/step - loss: 1.1258 - accuracy: 0.5673 - val_loss: 1.6319 - val_accuracy: 0.2593
Epoch 7/10
52/52 [==============================] - 0s 5ms/step - loss: 1.0484 - accuracy: 0.6442 - val_loss: 1.5803 - val_accuracy: 0.2593
Epoch 8/10
52/52 [=

In [8]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')


2/2 [==============================] - 0s 8ms/step - loss: 1.2263 - accuracy: 0.5091
Accuracy: 50.91%


In [9]:
history_dict = history.history
history_dict.keys()
#plot for accuracy and loss 

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# پیش‌بینی
new_text = input()
encoded_new_text = np.array([multi_hot_encode(new_text, vocab)])
prediction = model.predict(encoded_new_text)
predicted_index = np.argmax(prediction)
print(f'Predicted emoji index: {predicted_index}')


In [ ]:
emoji_dict = {"❤":0 , "⚾" : 1 , "😀":2 , "😔" : 3 , "🍴":4}
for key , value in emoji_dict.items():
    if predicted_index==value:
        print(key)

# BOW

In [20]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv("train_emoji.csv")
test = pd.read_csv("tesss.csv")
x_train = train.iloc[: , 0]
y_train = train.iloc[: , 1]
x_test = test.iloc[: , 0]
y_test = test.iloc[: , 1]

x_train = x_train.tolist()
y_train = y_train.tolist()
x_test = x_test.tolist()
y_test = y_test.tolist()

# تبدیل متن به بردارهای BOW
vectorizer = CountVectorizer()
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.transform(x_test)

# تبدیل لیبل‌ها به آرایه numpy
y_train = np.array(y_train)
y_test = np.array(y_test)

# تعداد کلاس‌ها (ایموجی‌ها)
num_classes = len(set(y_train) | set(y_test))


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# ساخت مدل
model = Sequential()
model.add(Dense(128, input_dim=X_train_bow.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# کامپایل مدل
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# آموزش مدل
history = model.fit(x_train_bow.toarray(), y_train, epochs=10, 
                    batch_size=32, validation_data=(x_test_bow.toarray(), y_test))


Epoch 1/10
5/5 [==============================] - 2s 167ms/step - loss: 1.5821 - accuracy: 0.2595 - val_loss: 1.5644 - val_accuracy: 0.3091
Epoch 2/10
5/5 [==============================] - 0s 23ms/step - loss: 1.5195 - accuracy: 0.3282 - val_loss: 1.5370 - val_accuracy: 0.3091
Epoch 3/10
5/5 [==============================] - 0s 23ms/step - loss: 1.4672 - accuracy: 0.3740 - val_loss: 1.5110 - val_accuracy: 0.3091
Epoch 4/10
5/5 [==============================] - 0s 24ms/step - loss: 1.4138 - accuracy: 0.4275 - val_loss: 1.4892 - val_accuracy: 0.2909
Epoch 5/10
5/5 [==============================] - 0s 24ms/step - loss: 1.3628 - accuracy: 0.5115 - val_loss: 1.4685 - val_accuracy: 0.3455
Epoch 6/10
5/5 [==============================] - 0s 23ms/step - loss: 1.3095 - accuracy: 0.5878 - val_loss: 1.4428 - val_accuracy: 0.3636
Epoch 7/10
5/5 [==============================] - 0s 24ms/step - loss: 1.2508 - accuracy: 0.6336 - val_loss: 1.4153 - val_accuracy: 0.4000
Epoch 8/10
5/5 [==========

In [24]:
# ارزیابی مدل روی داده‌های تست
loss, accuracy = model.evaluate(X_test_bow.toarray(), y_test)
print(f'Accuracy on test data: {accuracy * 100:.2f}%')



2/2 [==============================] - 0s 5ms/step - loss: 1.3232 - accuracy: 0.4000
Accuracy on test data: 40.00%


In [ ]:
# رسم نمودارهای loss و accuracy
import matplotlib.pyplot as plt

# رسم نمودار loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')
plt.show()

# رسم نمودار accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')
plt.show()


In [ ]:
def predict_emoji(text, model, vectorizer):
    # Transform the input text to BOW representation
    text_bow = vectorizer.transform([text])
    
    # Predict the emoji index
    predicted_probs = model.predict(text_bow.toarray())
    predicted_index = np.argmax(predicted_probs, axis=1)[0]
    
    return predicted_index

# Example usage:
text = input()
predicted_index = predict_emoji(text, model, vectorizer)
print(f'Predicted emoji index for "{text}": {predicted_index}')

In [ ]:
emoji_dict = {"❤":0 , "⚾" : 1 , "😀":2 , "😔" : 3 , "🍴":4}
for key , value in emoji_dict.items():
    if predicted_index==value:
        print(key)